# Animation of the Fourier sine series in Jupyter

Set up the environment

Todo:
- Axis labels and ticks

In [ ]:
%matplotlib inline

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches

from matplotlib import animation, rc
from IPython.display import HTML

import matplotlib
matplotlib.rcParams['animation.embed_limit'] = 2**128

## Creating the Animation
First set up the figure, the axis, and the plot element we want to animate:

In [ ]:
fig = plt.figure()
ax1 = plt.axes()
line, = ax1.plot([], [], lw=2)
plt.xlabel('x')
plt.ylabel('y')

ax1.set_xlim((-3.5, 10))
ax1.set_ylim((-3, 3.5))

plotlays, plotcols = [5], ["black", "red", "cyan", "blue", "magenta"]

l_width = [2, 2, 1, 2, 1]
lines = []
for index in range(5):
    lobj = ax1.plot([],[],lw=l_width[index],color=plotcols[index])[0]
    lines.append(lobj)
    

Define the initialization function, which plots the background of each frame:

In [ ]:
def init():
    for line in lines:
        line.set_data([],[])
    return lines

## Define rotating arrows

We are defining rotating arrows through sine and cosine functions. The variation in the x direction through the cosine functions is only done for illustration purposes to nicely show the wipers and circles.

In [ ]:
def arrow(start_point, radius, frequency, offset, i):
    r = np.linspace(0, radius, 100)
    x = start_point[0] + r * np.cos(frequency * i + offset)
    y = start_point[1] + r * np.sin(frequency * i + offset)

    return x, y

Define the animation function, which is called for each new frame:

In [ ]:
# Add two list for the history of the tip of the last arrow
x_hist = []
y_hist = []

x_drawn = []
y_drawn = []

x_org = []
y_org = []

num_harmonics = 3

def animate(i):
    # Define the Fourier sine coefficients
    harmonics = []
    for j in range(num_harmonics):
        # Append: radius, frequency and offset
        # You can comment the command for the square wave and uncomment the one for the sawtooth
        # Square wave
        harmonics.append([4/((2*j + 1) * np.pi), 2*j+1, 0.0])
        
        # Sawtooth
        #harmonics.append([2*(-1)**(j+2)/((j+1) * np.pi), j+1, 0.0])

    # Calculate the wipers and circles for the harmonics
    wipers_x = []
    wipers_y = []
    circles_x = []
    circles_y = []
    phi = np.linspace(0, 3*np.pi, 100)
    start = [0, 0]
    for params in harmonics:    
        x, y = arrow(start, params[0], 2 * np.pi*params[1]/50, params[2], i)
        wipers_x = [*wipers_x, *x]
        wipers_y = [*wipers_y, *y]
        cir_x = start[0] + params[0] * np.cos(phi)
        cir_y = start[1] + params[0] * np.sin(phi)
        start = [x[-1], y[-1]]
        circles_x.append([cir_x])
        circles_y.append([cir_y])
    
    # Plot the wipers
    lines[0].set_data(wipers_x, wipers_y)
    
    # Trace of the 2D shape
    x_hist.append(start[0])
    y_hist.append(start[1])
    joined_x = [*x_hist[-1:-200:-1]] 
    joined_y = [*y_hist[-1:-200:-1]] 
    lines[1].set_data(joined_x, joined_y)
    
    # Connecting the wipers and projection
    x_connect = np.linspace(start[0], 4, 100)
    y_connect = start[1] * np.ones(100)
    lines[2].set_data(x_connect, y_connect)
    
    # Projection onto the imaginary axis
    x_drawn.append(3.95)
    x_drawn[:] = [x + 0.05 for x in x_drawn]
    y_drawn.append(start[1])
    lines[3].set_data(x_drawn[:], y_drawn[:])

    # Plot original function
    # Hard-coded for the square wave and the given interval
    x_org.append(3.95)
    x_org[:] = [x + 0.05 for x in x_drawn]
    if i % 51 <= 25:
        tmp = 1
    else:
        tmp = -1
    y_org.append(tmp)
    lines[4].set_data(x_org[:], y_org[:])
    
    # Plot circles around the wipers
    jj = 5
    for x, y in zip(circles_x, circles_y):
        lobj = ax1.plot([],[],lw=0.5,color="black")[0]
        lines.append(lobj)  
        lines[jj].set_data(x, y)
        jj += 1
   
    return lines

Compile the animation. Setting blit=True will only re-draw the parts that have changed.

In [ ]:
anim = animation.FuncAnimation(fig, animate, init_func=init,
                               frames=100, interval=100, 
                               blit=True)

## Displaying the Animation

Now to create an interactive JavaScript widget using the to_jshtml method:

In [ ]:
# Inline animation
HTML(anim.to_jshtml())

In [ ]:
# Save the animation as a website
Writer = animation.writers['html']
writer = Writer(fps=60)
anim.save('square_wave.html', writer=writer, dpi=150)